<a href="https://colab.research.google.com/github/Sohaib9920/Transformer_Multilingual_NER/blob/main/Transformer_Multilingual_Named_Entity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Task:** Perform NER for a customer based in Switzerland, where there are four national languages: German (62.9%), French (22.9%), Italian (8.4%), and English (5.9%)

In [37]:
!pip install -U datasets
!pip install -U transformers
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 37.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.0 MB/s eta 0:00:00


In [28]:
import datasets
import torch
import transformers
from transformers import XLMRobertaConfig, RobertaPreTrainedModel, RobertaModel
from transformers.modeling_outputs import TokenClassifierOutput
from collections import defaultdict
import pandas as pd
from collections import Counter

# Dataset

We will be using a subset of the Cross-lingual TRansfer Evaluation of
Multilingual Encoders (XTREME) benchmark called WikiANN or PAN-X

In [2]:
xtreme_subsets = datasets.get_dataset_config_names("xtreme")
PAN_subsets = [s for s in xtreme_subsets if s.startswith("P")]
print(PAN_subsets)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for xtreme contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xtreme
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg', 'PAN-X.bn', 'PAN-X.de', 'PAN-X.el', 'PAN-X.en', 'PAN-X.es', 'PAN-X.et', 'PAN-X.eu', 'PAN-X.fa', 'PAN-X.fi', 'PAN-X.fr', 'PAN-X.he', 'PAN-X.hi', 'PAN-X.hu', 'PAN-X.id', 'PAN-X.it', 'PAN-X.ja', 'PAN-X.jv', 'PAN-X.ka', 'PAN-X.kk', 'PAN-X.ko', 'PAN-X.ml', 'PAN-X.mr', 'PAN-X.ms', 'PAN-X.my', 'PAN-X.nl', 'PAN-X.pt', 'PAN-X.ru', 'PAN-X.sw', 'PAN-X.ta', 'PAN-X.te', 'PAN-X.th', 'PAN-X.tl', 'PAN-X.tr', 'PAN-X.ur', 'PAN-X.vi', 'PAN-X.yo', 'PAN-X.zh', 'PAWS-X.de', 'PAWS-X.en', 'PAWS-X.es', 'PAWS-X.fr', 'PAWS-X.ja', 'PAWS-X.ko', 'PAWS-X.zh']


In [3]:
langs = ["de", "fr", "it", "en"]
props = [0.629, 0.229, 0.084, 0.058]
panx = defaultdict(dict)
for lang, prop in zip(langs, props):
    data_dict = datasets.load_dataset("xtreme", name=f"PAN-X.{lang}")
    for split in data_dict:
        selected_size = int(prop * data_dict[split].num_rows)
        panx[lang][split] = data_dict[split].shuffle(seed=42).select(range(selected_size))
        panx[lang] = datasets.DatasetDict(panx[lang])

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for xtreme contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xtreme
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
df = pd.DataFrame({lang: panx[lang].num_rows for lang in langs})
df

,de,fr,it,en
train,12580,4580,1680,1160
validation,6290,2290,840,580
test,6290,2290,840,580


In [5]:
df.sum(axis=1)

train         20000
validation    10000
test          10000
dtype: int64

In [6]:
tags = panx["de"]["train"].features["ner_tags"].feature
tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)

Each article is annotated with LOC (location), PER
(person), and ORG (organization) tags in the “inside-outside-beginning” (IOB2) format. In this format, a B-prefix indicates the beginning of an entity, and consecutive
tokens belonging to the same entity are given an I-prefix. An O tag indicates that the
token does not belong to any entity

In [7]:
panx_de = panx["de"].map(lambda batch: {"ner_tags_str":tags.int2str(batch["ner_tags"])})

In [8]:
pd.set_option("display.max_colwidth", None)
pd.DataFrame(panx_de["train"]).head(5)

,tokens,ner_tags,langs,ner_tags_str
0,"[Olympique, Nîmes, ,, Auxerres, seinerzeitiger, drittklassiger, Endspielgegner, ,, hatte, sich, erst, gar, nicht, für, die, Hauptrunde, qualifizieren, können, .]","[3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[de, de, de, de, de, de, de, de, de, de, de, de, de, de, de, de, de, de, de]","[B-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,"[Alexander, Jakowlew, wies, deshalb, am, 18, .]","[1, 2, 0, 0, 0, 0, 0]","[de, de, de, de, de, de, de]","[B-PER, I-PER, O, O, O, O, O]"
2,"[Aktuell, werden, die, beiden, am, besten, erhaltenen, Tafeln, im, Alten, Museum, ausgestellt, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0]","[de, de, de, de, de, de, de, de, de, de, de, de, de]","[O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, O, O]"
3,"[Sie, gehören, damit, mit, der, Dickschnabellumme, zu, den, schwersten, ,, derzeit, noch, lebenden, Alkenarten, .]","[0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[de, de, de, de, de, de, de, de, de, de, de, de, de, de, de]","[O, O, O, O, O, B-LOC, O, O, O, O, O, O, O, O, O]"
4,"[Auch, in, Garmisch-Partenkirchen, landete, er, auf, diesem, Platz, .]","[0, 0, 5, 0, 0, 0, 0, 0, 0]","[de, de, de, de, de, de, de, de, de]","[O, O, B-LOC, O, O, O, O, O, O]"


In [9]:
df = pd.DataFrame()
for split in panx_de:
    count = Counter()
    for row in panx_de[split]["ner_tags_str"]:
        count.update(row)
    df[split] = dict(count)

In [10]:
df.T

,B-ORG,I-ORG,O,B-PER,I-PER,B-LOC,I-LOC
train,5397,7647,86156,5881,8423,6169,2898
validation,2639,3774,43122,2870,4140,3172,1447
test,2657,3825,43170,2971,4260,3100,1404


This looks good—the distributions of the PER, LOC, and ORG frequencies are roughly
the same for each split, so the validation and test sets should provide a good measure
of our NER tagger’s ability to generalize.

# Tokenization

In [11]:
bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = transformers.AutoTokenizer.from_pretrained(xlmr_model_name)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [74]:
text = "Transformer models are cool n awsome!"
print(bert_tokenizer(text).tokens())
print(xlmr_tokenizer(text).tokens())

['[CLS]', 'Trans', '##former', 'models', 'are', 'cool', 'n', 'a', '##ws', '##ome', '!', '[SEP]']
['<s>', '▁Trans', 'former', '▁models', '▁are', '▁cool', '▁n', '▁a', 'w', 'some', '!', '</s>']


XML-R uses SentencePiece instead of WordPiece used by BERT. It is suitable for multilingual corpora.

# Creating a Custom Model for Token Classification

Imagine that `XLMRobertaForTokenClassification` do not exist so we will create it. `RobertaModel` has same architecture as desired by `XMLRobertaModel` so we will use it.

In [13]:
# Using Model.from_pretrained(pretrained_model_name_or_path, **kwargs) where Model is Derived Pretrained Model with config_class attribute and init method defined
# 1. Download configuration from config path if not given using model.config_class.from_pretrained.
#    Give warning if model_type in config class, which is derived PretrainedConfig, is not the same as given in loaded config.
#    kwargs accepted by model are used and unused kwargs are returned as model_kwargs
#    config.name_or_path = pretrained_model_name_or_path
# 2. Download the weights file from pretrained_model_name_or_path with name 'pytorch_model.bin' or 'model.safetensors' etc
# 3. Load state_dict from it
# 4. Instantiate the model with given config and model_kwargs / model_args.
#    It will instantiate the PreTrainedModel which will do self.config = config and self.name_or_path = config.name_or_path and instantiate nn.Module
#    Weights are initialized in model instantiation using self.post_init() which uses _init function defined in RobertaPreTrainedModel
# 5. Check the loaded_keys (from loaded state_dict) and expected_keys (from existing state_dict) and base_model_prefix attribute of model
# 6. Override the existing weights with loaded weights. Make sure that base model is creted as self.base_model_prefix.
# Note: config_class is optional. It is required to tell AutoModel which base model to instantiate as indicted by its 'model_type' attribute.

In [29]:
class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig
    base_model_prefix = "roberta"
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = torch.nn.Dropout(classifier_dropout)
        self.classifier = torch.nn.Linear(config.hidden_size, config.num_labels)
        self.post_init()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        outputs = self.roberta(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, **kwargs)
        # outputs contain hidden_states if kwargs contain output_hidden_states=True or self.config.output_hidden_states=True.
        # Same for other optional model outputs
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), (labels.view(-1)))
        return TokenClassifierOutput(
            loss = loss,
            logits = logits,
            hidden_states = outputs.hidden_states,
            attentions = outputs.attentions
        )

In [48]:
id2label = {id:tag for id, tag in enumerate(tags.names)}
label2id = {tag:id for id, tag in enumerate(tags.names)}

In [50]:
print(id2label)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}


In [95]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xlmr_model = (XLMRobertaForTokenClassification
              .from_pretrained(xlmr_model_name, id2label=id2label, label2id=label2id, num_labels=tags.num_classes)
              .to(device))

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [99]:
xlmr_model.config

XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 5,
    "B-ORG": 3,
    "B-PER": 1,
    "I-LOC": 6,
    "I-ORG": 4,
    "I-PER": 2,
    "O": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.37.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

In [88]:
def tag_text(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt")
    tokens = inputs.tokens()
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs.to(device))
    logits = outputs.logits[0]
    preds = torch.argmax(logits, axis=-1).cpu().numpy()
    pred_label = [model.config.id2label[id] for id in preds]
    return pd.DataFrame([tokens, pred_label], index=["Tokens", "Pred_Tags"])

In [90]:
text = "Jack Sparrow loves New York!"
tag_text(text, xlmr_tokenizer, xlmr_model)

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
Pred_Tags,B-ORG,I-ORG,I-ORG,I-ORG,B-PER,B-PER,B-PER,I-ORG,I-ORG,B-ORG


In [108]:
example = panx_de["train"][0]
tokens = example["tokens"]
ner_tags = example["ner_tags"]
ner_tags_str = example["ner_tags_str"]

In [115]:
print(tokens)

['Olympique', 'Nîmes', ',', 'Auxerres', 'seinerzeitiger', 'drittklassiger', 'Endspielgegner', ',', 'hatte', 'sich', 'erst', 'gar', 'nicht', 'für', 'die', 'Hauptrunde', 'qualifizieren', 'können', '.']


In [163]:
tokenized = xlmr_tokenizer(tokens, is_split_into_words=True)
tokenized_tokens = tokenized.tokens()
word_ids = tokenized.word_ids()

In [123]:
import numpy as np

In [159]:
subword_ner_tags = np.array(ner_tags)[word_ids[1:-1]]
subword_ner_tags_cleaned = np.where(pd.Series(subword_ner_tags).duplicated(), 0, subword_ner_tags)
subword_ner_tags_cleaned_str = ["<s>"] + list(np.array(tags.names)[subword_ner_tags_cleaned]) + ["</s>"]

In [160]:
pd.DataFrame([tokenized_tokens, word_ids, subword_ner_tags], index=["tokens", "word_ids", "subword_ner_tags", ])

In [162]:
print(subword_ner_tags_cleaned_str)

['<s>', 'B-ORG', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '</s>']
